In [1]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Custom training with custom training container and automatic registering of the model

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/custom/custom_training_container_and_model_registry.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fofficial%2Fcustom%2Fcustom_training_container_and_model_registry.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/custom/custom_training_container_and_model_registry.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/custom/custom_training_container_and_model_registry.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

## Overview


This tutorial demonstrates how to use the Vertex AI SDK to train using a custom container and automatically register the model.

Learn more about [Custom training](https://cloud.google.com/vertex-ai/docs/training/custom-training)

### Objective

In this tutorial, you create a custom model from a Python script in a custom Docker container using the Vertex AI SDK, and automatically register the model in the Vertex AI Model Registry. You can alternatively create custom models using `gcloud` command-line tool or online using Cloud Console.

This tutorial uses the following Google Cloud ML services and resources:

- Vertex AI Model Registry
- Vertex AI Training


The steps performed include:

- Create a Vertex AI custom job for training a model.
- Train and register a TensorFlow model using a custom container,
- List the registered model from the Vertex AI Model Registry.

### Dataset

The dataset used for this tutorial is the [CIFAR10 dataset](https://www.tensorflow.org/datasets/catalog/cifar10) from [TensorFlow Datasets](https://www.tensorflow.org/datasets/catalog/overview). The version of the dataset you will use is built into TensorFlow. The trained model predicts which type of class an image is from ten classes: airplane, automobile, bird, cat, deer, dog, frog, horse, ship, truck.

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage

Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

## Get started

### Install Vertex AI SDK for Python and other required packages


In [2]:
! pip3 install --upgrade google-cloud-aiplatform --quiet

### Restart runtime (Colab only)

To use the newly installed packages, you must restart the runtime on Google Colab.

In [26]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Wait until it's finished before continuing to the next step. ⚠️</b>
</div>


### Authenticate your notebook environment (Colab only)

Authenticate your environment on Google Colab.


In [27]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information and initialize Vertex AI SDK for Python

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com). Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [28]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

#### Create a Cloud Storage bucket

Create a storage bucket to store intermediate artifacts such as datasets.

In [29]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

**If your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [30]:
! gsutil mb -l $LOCATION -p $PROJECT_ID $BUCKET_URI

Creating gs://your-bucket-name-vertexai-service-project-unique-nm12/...


#### Initialize Vertex AI SDK for Python

Initialize the Vertex AI SDK for Python for your project and corresponding bucket.

In [31]:
import google.cloud.aiplatform as aiplatform

aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

#### Set hardware accelerators

You can set hardware accelerators for training and prediction.

Set the variables `TRAIN_GPU/TRAIN_NGPU` to use a container image supporting a GPU and the number of GPUs allocated to the virtual machine (VM) instance. For example, to use a GPU container image with 4 Nvidia Telsa K80 GPUs allocated to each VM, you would specify:

    (aip.gapic.AcceleratorType.NVIDIA_TESLA_K80, 4)


Otherwise specify `(None, None)` to use a container image to run on a CPU.

Learn more [hardware accelerator support](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators) for your location

In [32]:
TRAIN_GPU, TRAIN_NGPU = (None, None)

#### Set pre-built containers

Set the pre-built Docker container image for prediction.

- Set the variable `TF` to the TensorFlow version of the container image. For example, `2-1` would be version 2.1, and `1-15` would be version 1.15. The following list shows some of the pre-built images available:


For the latest list, see [Pre-built containers for prediction](https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers).

In [33]:
TF = "2-7"

DEPLOY_VERSION = "tf2-cpu.{}".format(TF)
DEPLOY_IMAGE = "{}-docker.pkg.dev/vertex-ai/prediction/{}:latest".format(
    LOCATION.split("-")[0], DEPLOY_VERSION
)

print("Deployment:", DEPLOY_IMAGE)

Deployment: us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-7:latest


#### Set machine type

Next, set the machine type to use for training and prediction.

- Set the variables `TRAIN_COMPUTE` and `DEPLOY_COMPUTE` to configure  the compute resources for the VMs you will use for for training and prediction.
 - `machine type`
     - `n1-standard`: 3.75GB of memory per vCPU.
     - `n1-highmem`: 6.5GB of memory per vCPU
     - `n1-highcpu`: 0.9 GB of memory per vCPU
 - `vCPUs`: number of \[2, 4, 8, 16, 32, 64, 96 \]

*Note: The following is not supported for training:*

 - `standard`: 2 vCPUs
 - `highcpu`: 2, 4 and 8 vCPUs

*Note: You may also use n2 and e2 machine types for training and deployment, but they do not support GPUs*.

In [34]:
TRAIN_COMPUTE = "n1-standard-4"
print("Train machine type", TRAIN_COMPUTE)

Train machine type n1-standard-4


# Tutorial

Now you are ready to start creating your own custom model and training for CIFAR10.

## Train a model

There are two ways you can train a custom model using a container image:

- **Use a Google Cloud prebuilt container**. If you use a prebuilt container, you will additionally specify a Python package to install into the container image. This Python package contains your code for training a custom model.

- **Use your own custom container image**. If you use your own container, the container needs to contain your code for training a custom model.

### Create a Docker file

In this tutorial, you train a CIFAR10 model using your own custom container.

To use your own custom container, you build a Docker file. First, you will create a directory for the container components.

### Examine the training package

#### Package layout

Before you start the training, you will look at how a Python package is assembled for a custom training job. When unarchived, the package contains the following directory/file layout.

- PKG-INFO
- README.md
- setup.cfg
- setup.py
- trainer
  - \_\_init\_\_.py
  - task.py

The files `setup.cfg` and `setup.py` are the instructions for installing the package into the operating environment of the Docker image.

The file `trainer/task.py` is the Python script for executing the custom training job. *Note*, when we referred to it in the worker pool specification, we replace the directory slash with a dot (`trainer.task`) and dropped the file suffix (`.py`).

#### Package Assembly

In the following cells, you will assemble the training package.

In [35]:
# Make folder for Python training script

APPLICATION_DIR = "custom"
! rm -rf $APPLICATION_DIR
! mkdir $APPLICATION_DIR

# Add package information
! touch custom/README.md

setup_cfg = "[egg_info]\n\ntag_build =\n\ntag_date = 0"
! echo "$setup_cfg" > custom/setup.cfg

setup_py = "import setuptools\n\nsetuptools.setup(\n\n    install_requires=[\n\n        'tensorflow_datasets==1.3.0',\n\n    ],\n\n    packages=setuptools.find_packages())"
! echo "$setup_py" > custom/setup.py

pkg_info = "Metadata-Version: 1.0\n\nName: CIFAR10 image classification\n\nVersion: 0.0.0\n\nSummary: Demostration training script\n\nHome-page: www.google.com\n\nAuthor: Google\n\nAuthor-email: aferlitsch@google.com\n\nLicense: Public\n\nDescription: Demo\n\nPlatform: Vertex"
! echo "$pkg_info" > custom/PKG-INFO

# Make the training subfolder
! mkdir custom/trainer
! touch custom/trainer/__init__.py

#### Task.py contents

In the next cell, you write the contents of the training script task.py. We won't go into detail, it's just there for you to browse. In summary:

- Get the directory where to save the model artifacts from the command line (`--model_dir`), and if not specified, then from the environment variable `AIP_MODEL_DIR`.
- Loads CIFAR10 dataset from TF Datasets (tfds).
- Builds a model using TF.Keras model API.
- Compiles the model (`compile()`).
- Sets a training distribution strategy according to the argument `args.distribute`.
- Trains the model (`fit()`) with epochs and steps according to the arguments `args.epochs` and `args.steps`
- Saves the trained model (`save(args.model_dir)`) to the specified model directory.

In [36]:
%%writefile $APPLICATION_DIR/trainer/task.py
# Single, Mirror and Multi-Machine Distributed Training for CIFAR-10

import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.python.client import device_lib
import argparse
import os
import sys
tfds.disable_progress_bar()

PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique" 

parser = argparse.ArgumentParser()
parser.add_argument('--model-dir', dest='model_dir',
                    default= BUCKET_URI + "/model", type=str, help='Model dir.')
parser.add_argument('--lr', dest='lr',
                    default=0.01, type=float,
                    help='Learning rate.')
parser.add_argument('--epochs', dest='epochs',
                    default=10, type=int,
                    help='Number of epochs.')
parser.add_argument('--steps', dest='steps',
                    default=200, type=int,
                    help='Number of steps per epoch.')
parser.add_argument('--distribute', dest='distribute', type=str, default='single',
                    help='distributed training strategy')
args = parser.parse_args()
print('Python Version = {}'.format(sys.version))
print('TensorFlow Version = {}'.format(tf.__version__))
print('TF_CONFIG = {}'.format(os.environ.get('TF_CONFIG', 'Not found')))
print('DEVICES', device_lib.list_local_devices())

# Single Machine, single compute device
if args.distribute == 'single':
    if tf.test.is_gpu_available():
        strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")
    else:
        strategy = tf.distribute.OneDeviceStrategy(device="/cpu:0")
# Single Machine, multiple compute device
elif args.distribute == 'mirror':
    strategy = tf.distribute.MirroredStrategy()
# Multiple Machine, multiple compute device
elif args.distribute == 'multi':
    strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()

# Multi-worker configuration
print('num_replicas_in_sync = {}'.format(strategy.num_replicas_in_sync))

# Preparing dataset
BUFFER_SIZE = 10000
BATCH_SIZE = 64


def make_datasets_unbatched():

  # Scaling CIFAR10 data from (0, 255] to (0., 1.]
  def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.0
    return image, label


  datasets, info = tfds.load(name='cifar10',
                            with_info=True,
                            as_supervised=True)
  return datasets['train'].map(scale).cache().shuffle(BUFFER_SIZE).repeat()


# Build the Keras model
def build_and_compile_cnn_model():
  model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(32, 32, 3)),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Conv2D(32, 3, activation='relu'),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(10, activation='softmax')
  ])
  model.compile(
      loss=tf.keras.losses.sparse_categorical_crossentropy,
      optimizer=tf.keras.optimizers.SGD(learning_rate=args.lr),
      metrics=['accuracy'])
  return model


# Train the model
NUM_WORKERS = strategy.num_replicas_in_sync
# Here the batch size scales up by number of workers since
# `tf.data.Dataset.batch` expects the global batch size.
GLOBAL_BATCH_SIZE = BATCH_SIZE * NUM_WORKERS
train_dataset = make_datasets_unbatched().batch(GLOBAL_BATCH_SIZE)

with strategy.scope():
  # Creation of dataset, and model building/compiling need to be within
  # `strategy.scope()`.
  model = build_and_compile_cnn_model()

model.fit(x=train_dataset, epochs=args.epochs, steps_per_epoch=args.steps)
model.save(args.model_dir)

Writing custom/trainer/task.py


#### Write the Docker file contents

Your first step in containerizing your code is to create a Docker file. In your Docker you’ll include all the commands needed to run your container image. It’ll install all the libraries you’re using and set up the entry point for your training code.

1. Install a pre-defined container image from TensorFlow repository for deep learning images.
2. Copies in the Python training code, to be shown subsequently.
3. Sets the entry into the Python training script as `trainer/task.py`. Note, the `.py` is dropped in the ENTRYPOINT command, as it is implied.

In [37]:
%%writefile $APPLICATION_DIR/Dockerfile

FROM gcr.io/deeplearning-platform-release/tf2-cpu.2-3
WORKDIR /root

WORKDIR /

# Copies the trainer code to the docker image.
COPY trainer /trainer

# Sets up the entry point to invoke the trainer.
ENTRYPOINT ["python", "-m", "trainer.task"]

Writing custom/Dockerfile


### Enable Artifact Registry API

First, you must enable the Artifact Registry API service for your project.

Learn more about [enabling the Artifact Registry service](https://cloud.google.com/artifact-registry/docs/enable-service).

In [38]:
! gcloud services enable artifactregistry.googleapis.com

### Build the container

You store the Docker image in Artifact Registry. First, create a Docker repository in Artifact Registry

In [39]:
import os

REPO_NAME='cifar10'

if os.getenv("IS_TESTING"):
    ! sudo apt-get update --yes && sudo apt-get --only-upgrade --yes install google-cloud-sdk-cloud-run-proxy google-cloud-sdk-harbourbridge google-cloud-sdk-cbt google-cloud-sdk-gke-gcloud-auth-plugin google-cloud-sdk-kpt google-cloud-sdk-local-extract google-cloud-sdk-minikube google-cloud-sdk-app-engine-java google-cloud-sdk-app-engine-go google-cloud-sdk-app-engine-python google-cloud-sdk-spanner-emulator google-cloud-sdk-bigtable-emulator google-cloud-sdk-nomos google-cloud-sdk-package-go-module google-cloud-sdk-firestore-emulator kubectl google-cloud-sdk-datastore-emulator google-cloud-sdk-app-engine-python-extras google-cloud-sdk-cloud-build-local google-cloud-sdk-kubectl-oidc google-cloud-sdk-anthos-auth google-cloud-sdk-app-engine-grpc google-cloud-sdk-pubsub-emulator google-cloud-sdk-datalab google-cloud-sdk-skaffold google-cloud-sdk google-cloud-sdk-terraform-tools google-cloud-sdk-config-connector
    ! gcloud components update --quiet

!gcloud artifacts repositories create $REPO_NAME --repository-format=docker \
--location=$LOCATION --description="Docker repository"

Create request issued for: [cifar10]
Waiting for operation [projects/vertexai-service-project/locations/us-central1/
operations/e418a7fd-89cb-4162-a9e4-9cfc8d0f9a66] to complete...done.           
Created repository [cifar10].


### Configure authentication to your private repo

Before you push or pull container images, configure Docker to use the `gcloud` command-line tool to authenticate requests to Artifact Registry for your region.

In [40]:
! gcloud auth configure-docker {LOCATION}-docker.pkg.dev --quiet


{
  "credHelpers": {
    "gcr.io": "gcloud",
    "us.gcr.io": "gcloud",
    "eu.gcr.io": "gcloud",
    "asia.gcr.io": "gcloud",
    "staging-k8s.gcr.io": "gcloud",
    "marketplace.gcr.io": "gcloud",
    "us-central1-docker.pkg.dev": "gcloud"
  }
}
Adding credentials for: us-central1-docker.pkg.dev
gcloud credential helpers already registered correctly.


Define a variable with the URI of your Docker image in Artifact Registry:

In [41]:
IMAGE_URI = f"{LOCATION}-docker.pkg.dev/{PROJECT_ID}/{REPO_NAME}/cifar10:latest"

Navigate to the directory containing the Dockerfile

In [42]:
cd $APPLICATION_DIR

/home/jupyter/custom


Push the image to the created artifact repository using Cloud-Build.

**Note**: The following command automatically uses the Dockerfile from the directory it's being run from.

In [43]:
import sys

IS_COLAB = "google.colab" in sys.modules

if not IS_COLAB:
    ! docker build ./ -t $IMAGE_URI
    ! docker push $IMAGE_URI

else:
    ! gcloud builds submit --region={LOCATION} --tag={IMAGE_URI}

Sending build context to Docker daemon  11.26kB
Step 1/5 : FROM gcr.io/deeplearning-platform-release/tf2-cpu.2-3
 ---> 1fe9cc789976
Step 2/5 : WORKDIR /root
 ---> Using cache
 ---> 108c4d5ec707
Step 3/5 : WORKDIR /
 ---> Using cache
 ---> b1b6322a2063
Step 4/5 : COPY trainer /trainer
 ---> 6a76f8369953
Step 5/5 : ENTRYPOINT ["python", "-m", "trainer.task"]
 ---> Running in 9e684901f9a8
Removing intermediate container 9e684901f9a8
 ---> 3bc9d29af5e7
Successfully built 3bc9d29af5e7
Successfully tagged us-central1-docker.pkg.dev/vertexai-service-project/cifar10/cifar10:latest
The push refers to repository [us-central1-docker.pkg.dev/vertexai-service-project/cifar10/cifar10]

4c78f45d: Preparing 
27243e9e: Preparing 
7443c6ac: Preparing 
3b32694d: Preparing 
005f4efc: Preparing 
ce19c9dd: Preparing 
faab67e8: Preparing 
2b5f8a22: Preparing 
6cf8673e: Preparing 
2cf21221: Preparing 
24722d3f: Preparing 
b9d4bbb9: Preparing 
0533bd85: Preparing 
e1652f88: Preparing 
17ce4d28: Preparing 
70df

#### Store training script on your Cloud Storage bucket

Exit the directory containing the Dockerfile

In [47]:
cd ..

/home/jupyter


Next, package the training folder into a compressed tar ball, and then store the package in your Cloud Storage bucket.

In [48]:
! rm -f custom.tar custom.tar.gz
! tar cvf custom.tar custom
! gzip custom.tar
! gsutil cp custom.tar.gz $BUCKET_URI/trainer_cifar10.tar.gz

custom/
custom/trainer/
custom/trainer/task.py
custom/trainer/__init__.py
custom/Dockerfile
custom/setup.cfg
custom/PKG-INFO
custom/setup.py
custom/README.md
Copying file://custom.tar.gz [Content-Type=application/x-tar]...
/ [1 files][  2.2 KiB/  2.2 KiB]                                                
Operation completed over 1 objects/2.2 KiB.                                      


### Create and run custom training job


To train a custom model, you perform two steps: 
1) create a custom training job
2) run the job.

#### Create custom training job

A custom training job is created with the `CustomTrainingJob` class, with the following parameters:

- `display_name`: The human readable name for the custom training job.
- `container_uri`: The training container image.
- `model_serving_container_image_uri`: The container for deploying the model.

In [49]:
job = aiplatform.CustomContainerTrainingJob(
    display_name="cifar10",
    container_uri=IMAGE_URI,
    model_serving_container_image_uri=DEPLOY_IMAGE,
)

print(job)

### Prepare your command-line arguments

Now define the command-line arguments for your custom training container:

- `args`: The command-line arguments to pass to the executable that is set as the entry point into the container.
  - `--model-dir` : For our demonstrations, we use this command-line argument to specify where to store the model artifacts.
      - direct: You pass the Cloud Storage location as a command line argument to your training script (set variable `DIRECT = True`), or
      - indirect: The service passes the Cloud Storage location as the environment variable `AIP_MODEL_DIR` to your training script (set variable `DIRECT = False`). In this case, you tell the service the model artifact location in the job specification.
  - `"--epochs=" + EPOCHS`: The number of epochs for training.
  - `"--steps=" + STEPS`: The number of steps per epoch.

In [50]:
MODEL_DIR = BUCKET_URI

EPOCHS = 1
STEPS = 1

CMDARGS = [
    "--epochs=" + str(EPOCHS),
    "--steps=" + str(STEPS),
]

#### Run the custom training job

Next, you run the custom job to start the training job by invoking the method `run`, with the following parameters:

- `args`: The command-line arguments to pass to the training script.
- `replica_count`: The number of compute instances for training (replica_count = 1 is single node training).
- `machine_type`: The machine type for the compute instances.
- `accelerator_type`: The hardware accelerator type.
- `accelerator_count`: The number of accelerators to attach to a worker replica.
- `base_output_dir`: The Cloud Storage location to write the model artifacts to.
- `model_display_name`: A human readable name for the registered model.
- `sync`: Whether to block until completion of the job.

In [51]:
if TRAIN_GPU:
    model = job.run(
        args=CMDARGS,
        replica_count=1,
        machine_type=TRAIN_COMPUTE,
        accelerator_type=TRAIN_GPU.name,
        accelerator_count=TRAIN_NGPU,
        base_output_dir=MODEL_DIR,
        model_display_name="cifar10",
        sync=True,
    )
else:
    model = job.run(
        args=CMDARGS,
        replica_count=1,
        machine_type=TRAIN_COMPUTE,
        base_output_dir=MODEL_DIR,
        model_display_name="cifar10",
        sync=True,
    )

Training Output directory:
gs://your-bucket-name-vertexai-service-project-unique-nm12 
View Training:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/2495439255254335488?project=314481442207
CustomContainerTrainingJob projects/314481442207/locations/us-central1/trainingPipelines/2495439255254335488 current state:
PipelineState.PIPELINE_STATE_RUNNING
View backing custom job:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/2963167103663734784?project=314481442207
CustomContainerTrainingJob projects/314481442207/locations/us-central1/trainingPipelines/2495439255254335488 current state:
PipelineState.PIPELINE_STATE_RUNNING
CustomContainerTrainingJob projects/314481442207/locations/us-central1/trainingPipelines/2495439255254335488 current state:
PipelineState.PIPELINE_STATE_RUNNING
CustomContainerTrainingJob projects/314481442207/locations/us-central1/trainingPipelines/2495439255254335488 current state:
PipelineState.PIPELINE_STATE_RUN

### View the model in the Model Registry

Finally, the `run()` method will return the model that was automatically registered in the model registry.

In [52]:
print(model.gca_resource)

name: "projects/314481442207/locations/us-central1/models/4588896440918474752@1"
version_id: "1"
version_aliases: "default"
version_create_time {
  seconds: 1718630803
  nanos: 921980000
}
version_update_time {
  seconds: 1718631109
  nanos: 579947000
}
display_name: "cifar10"
predict_schemata {
}
metadata {
}
supported_export_formats {
  id: "custom-trained"
  exportable_contents: ARTIFACT
}
training_pipeline: "projects/314481442207/locations/us-central1/trainingPipelines/2495439255254335488"
container_spec {
  image_uri: "us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-7:latest"
}
artifact_uri: "gs://your-bucket-name-vertexai-service-project-unique-nm12/model"
supported_deployment_resources_types: DEDICATED_RESOURCES
supported_deployment_resources_types: SHARED_RESOURCES
supported_input_storage_formats: "jsonl"
supported_input_storage_formats: "bigquery"
supported_input_storage_formats: "csv"
supported_input_storage_formats: "tf-record"
supported_input_storage_formats: "tf-record-gz

# Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial.

In [53]:
# Delete artifact registry repo
! gcloud artifacts repositories delete $REPO_NAME --location $LOCATION --quiet

delete_job = True
delete_model = True
delete_application_directory = False
delete_bucket = False

# Delete job
if delete_job:
    try:
        job.delete()
    except Exception as e:
        print(e)

# Delete model
if delete_model:
    try:
        model.delete()
    except Exception as e:
        print(e)

if delete_bucket:
    ! gsutil -m rm -r $BUCKET_URI

# Delete application directory
if delete_application_directory:
    !cd .. && rm -rf $APPLICATION_DIR custom.tar.gz

# Delete bucket
if delete_bucket:
    ! gsutil rm -r $BUCKET_URI

Delete request issued for: [cifar10]
Waiting for operation [projects/vertexai-service-project/locations/us-central1/
operations/b5c57bc6-85fe-439d-a8b4-362bdda7bb2d] to complete...done.           
Deleted repository [cifar10].
Deleting CustomContainerTrainingJob : projects/314481442207/locations/us-central1/trainingPipelines/2495439255254335488
Delete CustomContainerTrainingJob  backing LRO: projects/314481442207/locations/us-central1/operations/4609888626153619456
CustomContainerTrainingJob deleted. . Resource name: projects/314481442207/locations/us-central1/trainingPipelines/2495439255254335488
Deleting Model : projects/314481442207/locations/us-central1/models/4588896440918474752
Delete Model  backing LRO: projects/314481442207/locations/us-central1/models/4588896440918474752/operations/42675654046515200
Model deleted. . Resource name: projects/314481442207/locations/us-central1/models/4588896440918474752
